In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")


In [23]:
df = pd.read_csv('all.csv')
df =  df.drop(['All','O+','O-','B+','B-','A-','AB+','AB-'], axis=1)
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')
df.head()

,A+
Date,
2017-01-01,17
2017-01-02,13
2017-01-03,14
2017-01-04,11
2017-01-05,12


In [24]:
df.head()

,A+
Date,
2017-01-01,17
2017-01-02,13
2017-01-03,14
2017-01-04,11
2017-01-05,12


In [25]:
train, test = df[:-4], df[-4:]

In [26]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)
len(train) 


1095

In [27]:
n_input = 6
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=4)

In [28]:
model = Sequential()
model.add(LSTM(1000, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit_generator(generator, epochs=180)

AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [29]:
pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=df[-n_input:].index, columns=['Prediction'])

df_predict.head(15)

In [ ]:
df.head(12)

In [ ]:
df_test = pd.concat([df,df_predict], axis=1)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df_test.index, df_test['A+'])
plt.plot(df_test.index, df_test['Prediction'], color='r')
plt.legend(loc='best', fontsize='xx-large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
pred_actual_rmse = rmse(df_test.iloc[-n_input:, [0]], df_test.iloc[-n_input:, [1]])
print("rmse: ", pred_actual_rmse)

In [ ]:
train = df

n_input = 4
n_features = 1

scaler.fit(train)
train = scaler.transform(train)

generator = TimeseriesGenerator(train,train, length=n_input, batch_size=4)

model.fit_generator(generator, epochs=180)

pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:,:],[[pred_list[i]]], axis = 1)

In [ ]:
from pandas.tseries.offsets import DateOffset
add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,5) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)
future_dates

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

df_proj = pd.concat([df,df_predict], axis=1)
df_proj.index.name = 'Date'


In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df_proj.index, df_proj['A+'])
plt.plot(df_proj.index, df_proj['Prediction'], color='r')
plt.legend(loc='best', fontsize='xx-large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
print(df_proj)
df_proj.index.name ='Date'
df_proj.reset_index(inplace=True)
print(df_proj)
df_proj['Date'].to_list()

In [ ]:
df_proj['Date']=df_proj['Date'].astype(str)
x= df_proj['Date'].to_list()
x[-14:]

In [ ]:
import pickle
pickle.dump(model, open('LSTMTS.pkl','wb'))

In [ ]:
model1 = pickle.load(open('LSTMTS.pkl','rb'))
print(df_predict)

In [ ]:
import joblib

joblib.dump(scaler, "A+data_transformer.joblib")
model.save("A+model.h5")

In [ ]:
from datetime import timedelta, date
EndDate = date.today() + timedelta(days=7)
